# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 01:02:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   47C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2020년 Hyundai Data**

In [5]:
lexicon_2020 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2020_hyundai.csv')

lexicon_2020.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,현대차,5380,매일경제,2020010106,SUVs stay popular in S. Korea accounting 44.3%...,http://premium.mk.co.kr/view.php?no=27455,[Pulse로 배우는 영작문-762] 'Pulse로 배우는 영작문'으로 영문 뉴스 ...,2020-01-02,6,121000,121000,118000,118000,556532,-0.020747,-1,0,작문 작문 영문 뉴스 도전 영문 뉴스 원문 바로가기 라이프스타일 변화 글로벌 자동차...


In [6]:
lexicon_2020[lexicon_2020['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2020.dropna(axis=0, inplace=True)
lexicon_2020 = lexicon_2020.reset_index(drop=True)
lexicon_2020.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
neg_lexicon_2020 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_neg_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_neg와의 Cosine Similarity 계산
for x in range(len(lexicon_2020['Tokenization'])):
    news_num = x+1
    for y in range(len(negative)):
        for z in range(len(list(set(lexicon_2020['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2020['Tokenization'][x].split())):
                    if lexicon_2020['Tokenization'][x].split()[w] == list(set(lexicon_2020['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2020['date'][x],
                    'news_num': news_num,
                    'KOSELF_neg_word': negative[y],
                    'news_word': list(set(lexicon_2020['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                neg_lexicon_2020 = neg_lexicon_2020.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2020['date'][x], negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z], ko_model.wv.similarity(negative[y], list(set(lexicon_2020['Tokenization'][x].split()))[z])))

***2020-01-02 Cosine Similarity between <과적> & <차량> : 0.5693626403808594
***2020-01-02 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2020-01-02 Cosine Similarity between <약화> & <강화> : 0.5141976475715637
***2020-01-02 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2020-01-02 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2020-01-02 Cosine Similarity between <둔화> & <급락> : 0.5491443276405334
***2020-01-02 Cosine Similarity between <둔화> & <하락> : 0.5260781049728394
***2020-01-02 Cosine Similarity between <둔화> & <감소> : 0.5349085330963135
***2020-01-02 Cosine Similarity between <약화> & <감소> : 0.551184892654419
***2020-01-02 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2020-01-02 Cosine Similarity between <하락> & <상승> : 0.7126480340957642
***2020-01-02 Cosine Similarity between <하락> & <급락> : 0.6878489255905151
***2020-01-02 Cosine Similarity between <하락> & <감소> : 0.575989305973053
***2020-01-02 Cosine Similarity between <과적> & <차량> : 0

In [11]:
neg_lexicon_2020

,date,news_num,KOSELF_neg_word,news_word,cosine_similarity,frequency
0,2020-01-02,1,과적,차량,0.569363,1
1,2020-01-02,1,둔화,증가,0.504186,1
2,2020-01-02,1,약화,강화,0.514198,1
3,2020-01-02,1,하락,증가,0.553474,1
4,2020-01-02,2,둔화,증가,0.504186,1
...,...,...,...,...,...,...
16206,2021-01-04,4237,약화,감소,0.551185,1
16207,2021-01-04,4237,하락,증가,0.553474,2
16208,2021-01-04,4237,하락,감소,0.575989,1
16209,2021-01-04,4238,과적,차량,0.569363,3


In [12]:
neg_lexicon_2020.to_csv('../../../../Code/Data/Test/Stock-Year/neg_hyundai_2020.csv', index=False)

In [13]:
a = list(set(list(neg_lexicon_2020['news_word'])))

a

['둔화',
 '증가',
 '좌절',
 '엄벌',
 '손해배상',
 '강화',
 '상승',
 '선박',
 '성공',
 '등락',
 '끝내',
 '주춤',
 '손괴',
 '앉음마툰',
 '폭락',
 '합선',
 '긍적적',
 '복원성',
 '탑차',
 '급락',
 '보상',
 '구난',
 '폭등',
 '약세',
 '증차',
 '강점',
 '업황',
 '경색',
 '급감',
 '결항',
 '차량',
 '추징',
 '비리',
 '호전',
 '플러스',
 '폭로',
 '견조',
 '위축',
 '금값',
 '누유',
 '유발',
 '엠앤티',
 '특혜',
 '선령',
 '우발',
 '보합',
 '급회전',
 '응우옌부뚱',
 '폐업',
 '의심',
 '위기',
 '반등',
 '실패',
 '불황',
 '쇠퇴',
 '오벤터스',
 '침체',
 '항행',
 '급유',
 '참패',
 '절상',
 '연루',
 '악화',
 '하락',
 '호황',
 '불안',
 '경기',
 '화물',
 '혼돈',
 '감소',
 '급등',
 '저하',
 '병세',
 '조처',
 '텍드림텍',
 '갓길',
 '약화',
 '역풍',
 '과속',
 '악재',
 '토히르',
 '부도',
 '논란',
 '운항']

In [14]:
b = list(set(list(neg_lexicon_2020[neg_lexicon_2020['cosine_similarity']>=0.7]['news_word'])))

b

['의심', '실패', '상승', '성공', '침체', '약화']

In [15]:
c = list(set(list(neg_lexicon_2020[neg_lexicon_2020['cosine_similarity']>=0.65]['news_word'])))

c

['의심', '실패', '상승', '긍적적', '성공', '조처', '침체', '약화', '급락']